In [44]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
plt.style.use('seaborn')

import yfinance as yf



In [45]:
indices=['^NYA','^IXIC','^XAX','^DJI','^RUT']

equities = ['MANU', 'GOOG', 'EVGO', 'TJX', 'AMZN']

In [46]:
start_date='2010-01-01'
end_date='2023-05-01'

indices_data = yf.download(indices, start=start_date, end=end_date)['Adj Close']


[*********************100%***********************]  5 of 5 completed


In [47]:

equities_data = yf.download(equities, start=start_date, end=end_date)['Adj Close']

[*********************100%***********************]  5 of 5 completed


In [48]:
indices_returns = indices_data.pct_change()
equities_returns = equities_data.pct_change()


In [49]:
indices_cumulative_returns = (1 + indices_returns).cumprod()
equities_cumulative_returns = (1 + equities_returns).cumprod()


In [50]:
print(indices_cumulative_returns)


                ^DJI     ^IXIC      ^NYA      ^RUT      ^XAX
Date                                                        
2010-01-04       NaN       NaN       NaN       NaN       NaN
2010-01-05  0.998872  1.000126  1.003839  0.997485  1.003377
2010-01-06  0.999029  0.996825  1.006955  0.996641  1.007143
2010-01-07  1.002164  0.996374  1.009171  1.002921  1.007747
2010-01-08  1.003234  1.003790  1.013459  1.006968  1.010164
...              ...       ...       ...       ...       ...
2023-04-24  3.200636  5.214476  2.130103  2.794673  2.311039
2023-04-25  3.168080  5.111358  2.096912  2.727621  2.261110
2023-04-26  3.146447  5.135266  2.078493  2.703343  2.251842
2023-04-27  3.195983  5.259979  2.106208  2.735854  2.260932
2023-04-28  3.221683  5.296515  2.121800  2.763615  2.265518

[3353 rows x 5 columns]


In [51]:
print(equities_cumulative_returns)

                 AMZN      EVGO      GOOG      MANU        TJX
Date                                                          
2010-01-04        NaN       NaN       NaN       NaN        NaN
2010-01-05   1.005900       NaN  0.995596       NaN   1.026914
2010-01-06   0.987677       NaN  0.970499       NaN   1.031076
2010-01-07   0.970874       NaN  0.947906       NaN   1.083795
2010-01-08   0.997162       NaN  0.960542       NaN   1.069645
...               ...       ...       ...       ...        ...
2023-04-24  15.864077  0.617674  6.840382  1.658924  10.365932
2023-04-25  15.320388  0.588115  6.701371  1.618087  10.142853
2023-04-26  15.680359  0.603404  6.691121  1.584954  10.224693
2023-04-27  16.403286  0.620732  6.942239  1.587266  10.420053
2023-04-28  15.750559  0.610539  6.932629  1.544887  10.404212

[3353 rows x 5 columns]


In [52]:
def calculate_max_drawdown(data):
    cumulative_returns = (1 + data).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns / peak) - 1
    max_drawdown = drawdown.min()
    return max_drawdown

indices_max_drawdowns = calculate_max_drawdown(indices_returns)
equities_max_drawdowns = calculate_max_drawdown(equities_returns)


In [53]:
print(indices_max_drawdowns)
print(equities_max_drawdowns)

^DJI    -0.370862
^IXIC   -0.363953
^NYA    -0.381142
^RUT    -0.430613
^XAX    -0.535116
dtype: float64
AMZN   -0.561453
EVGO   -0.833711
GOOG   -0.446018
MANU   -0.580516
TJX    -0.425535
dtype: float64


In [54]:
risk_free_rate = 3.5  # Set the risk-free rate

def calculate_sharpe_ratio(data, risk_free_rate):
    excess_returns = data - risk_free_rate
    annualized_returns = excess_returns.mean() * 252  # Assuming 252 trading days in a year
    annualized_volatility = data.std() * np.sqrt(252)
    sharpe_ratio = annualized_returns / annualized_volatility
    return sharpe_ratio

def calculate_sortino_ratio(data, risk_free_rate):
    downside_returns = data.copy()
    downside_returns[downside_returns > 0] = 0  # Consider only negative returns for downside
    annualized_returns = data.mean() * 252
    downside_deviation = downside_returns.std() * np.sqrt(252)
    sortino_ratio = (annualized_returns - risk_free_rate) / downside_deviation
    return sortino_ratio

indices_sharpe_ratio = calculate_sharpe_ratio(indices_returns, risk_free_rate)
indices_sortino_ratio = calculate_sortino_ratio(indices_returns,risk_free_rate)


In [55]:
equities_sharpe_ratio=calculate_sharpe_ratio(equities_returns,risk_free_rate)
equities_sortino_ratio=calculate_sortino_ratio(equities_returns,risk_free_rate)

In [56]:
print(equities_sharpe_ratio)
print(equities_sortino_ratio)

AMZN   -2657.446054
EVGO    -920.185636
GOOG   -3225.721839
MANU   -2634.037338
TJX    -3427.656832
dtype: float64
AMZN   -16.545775
EVGO    -7.396731
GOOG   -20.343668
MANU   -17.840893
TJX    -21.366798
dtype: float64


In [57]:
print(indices_sharpe_ratio)
print(indices_sortino_ratio)

^DJI    -5132.742917
^IXIC   -4279.694433
^NYA    -5024.736170
^RUT    -3833.804743
^XAX    -4499.837851
dtype: float64
^DJI    -30.384165
^IXIC   -25.460691
^NYA    -29.832512
^RUT    -23.230882
^XAX    -26.981209
dtype: float64
